In [1]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import autograd
from torch.utils import data
from torch.optim import Adam
from torchvision import datasets, transforms
import torchvision.models as models

In [2]:
# 난수 생성기가 항상 일정한 값을 출력하게 하기 위해 seed 고정
random_seed = 2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

## EDA

In [3]:
DATASET_PATH = os.path.join('/USER/Taeyun/Traffic_TimeSeires')

In [4]:
train_df =pd.read_csv(os.path.join(DATASET_PATH, 'EDA_train.csv'))

In [5]:
train_df.describe()

,날짜,시간,10,100,101,120,121,140,150,160,...,1040,1100,1200,1510,2510,3000,4510,5510,6000,holiday
count,3.312000e+03,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,...,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000
mean,2.020030e+07,11.500000,231985.690142,66764.110482,7152.941450,17338.681788,6642.280696,4687.068715,99423.298536,4397.351122,...,10927.634008,27211.961177,13940.332050,4872.764694,14645.652929,2461.221518,12113.330289,7961.737872,13977.839598,0.065217
std,1.323436e+02,6.923232,121659.654634,41552.808761,5559.123832,13285.051885,4778.391378,3524.762551,58307.736503,2849.158325,...,6153.677817,15573.278037,7108.477643,4069.583305,9548.717571,1871.908504,7983.499271,5202.413198,9262.098817,0.246946
min,2.020010e+07,0.000000,18729.000000,3927.000000,313.000000,937.000000,229.000000,197.000000,7095.000000,222.000000,...,746.000000,1220.000000,1040.000000,246.000000,701.000000,45.000000,581.000000,358.000000,685.000000,0.000000
25%,2.020020e+07,5.750000,112291.250000,23691.500000,1913.000000,5229.250000,1706.250000,1328.000000,43342.500000,1541.250000,...,4544.000000,11649.250000,7355.250000,1505.750000,4997.250000,717.500000,3622.250000,2560.500000,3989.000000,0.000000
50%,2.020031e+07,11.500000,267492.500000,74079.000000,6950.500000,16645.500000,7039.000000,4710.000000,111840.500000,4757.000000,...,12878.500000,31122.000000,15922.500000,4258.000000,15928.500000,2201.500000,13522.500000,8567.500000,15190.500000,0.000000
75%,2.020041e+07,17.250000,327058.750000,99830.250000,10884.750000,25890.500000,10318.250000,6926.500000,140797.000000,6661.000000,...,16191.500000,39071.500000,19431.500000,6754.500000,21729.000000,3748.250000,18701.500000,12494.500000,22059.750000,0.000000
max,2.020052e+07,23.000000,580098.000000,200808.000000,45869.000000,114833.000000,24596.000000,24639.000000,292004.000000,14163.000000,...,25891.000000,76629.000000,35192.000000,28531.000000,45531.000000,10581.000000,38061.000000,21229.000000,36915.000000,1.000000


In [6]:
valid_df=pd.read_csv(os.path.join(DATASET_PATH, 'EDA_valid.csv'))

In [7]:
valid_df.describe()

,날짜,시간,10,100,101,120,121,140,150,160,...,1040,1100,1200,1510,2510,3000,4510,5510,6000,holiday
count,3.360000e+02,336.00000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.0
mean,2.020052e+07,11.50000,235737.547619,71831.784722,7599.669147,19486.676587,7092.030258,5144.850694,102989.622024,4617.031746,...,12064.953373,25553.853175,14523.321429,5280.790675,16045.720238,2404.031746,13443.192460,8990.027778,15001.383929,0.0
std,4.037141e+00,6.93251,117241.545464,43030.551231,5280.224111,13011.336904,4853.269352,3561.189715,56194.988032,2859.589709,...,6558.970811,13780.299649,7040.895436,3699.333684,9907.036864,1669.017296,8247.429868,5588.240572,9565.833154,0.0
min,2.020051e+07,0.00000,25263.000000,7113.000000,546.000000,1761.000000,392.000000,370.000000,11640.000000,428.000000,...,1313.000000,2574.000000,1931.000000,549.000000,1265.000000,94.000000,1136.000000,819.000000,1250.000000,0.0
25%,2.020051e+07,5.75000,125006.250000,26646.250000,2323.000000,6383.250000,2169.750000,1580.750000,50842.000000,1854.250000,...,5316.000000,11519.500000,8720.250000,1870.500000,5879.250000,799.916667,4511.500000,3311.250000,4696.000000,0.0
50%,2.020052e+07,11.50000,291569.500000,84219.500000,8047.000000,20308.000000,7731.000000,5582.000000,119572.000000,5062.000000,...,14996.500000,29671.000000,17037.500000,4891.000000,17802.500000,2363.500000,15901.500000,9662.500000,16447.000000,0.0
75%,2.020052e+07,17.25000,326878.250000,104994.750000,11640.875000,28508.250000,10776.750000,7523.750000,144746.250000,6806.250000,...,17698.500000,37066.500000,19961.750000,7417.000000,23388.750000,3497.500000,20104.250000,13849.250000,23266.250000,0.0
max,2.020052e+07,23.00000,439173.000000,150733.000000,19038.000000,53098.000000,17634.000000,14861.000000,226740.000000,10638.000000,...,21257.000000,48132.000000,26537.000000,15578.000000,36225.000000,6220.000000,29441.000000,19906.000000,31504.000000,0.0


In [8]:
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'EDA_test.csv'))

In [9]:
test_df.describe()

,날짜,시간,10,100,101,120,121,140,150,160,...,1040,1100,1200,1510,2510,3000,4510,5510,6000,holiday
count,3.360000e+02,336.00000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,...,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.000000,336.0
mean,2.020052e+07,11.50000,118874.625000,36042.446429,3346.947917,9309.401786,3084.322917,2074.049107,51864.145833,1835.991071,...,5596.458333,11984.901786,6884.318452,2203.113095,7539.523810,602.684524,6290.181548,4077.125000,7107.592262,0.0
std,4.037141e+00,6.93251,146557.491977,48467.781982,5803.180362,13919.548597,5381.828622,3984.745475,66753.570506,3508.836521,...,8089.586941,16127.871300,9343.563633,4226.126132,11090.738373,1931.305348,9381.722580,6501.804851,10648.331435,0.0
min,2.020052e+07,0.00000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.0
25%,2.020052e+07,5.75000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,...,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.0
50%,2.020052e+07,11.50000,12132.000000,3057.000000,-226.500000,483.000000,-295.500000,-308.500000,5320.500000,-280.500000,...,157.000000,787.500000,466.000000,-225.000000,133.000000,-452.500000,68.500000,-90.000000,125.500000,0.0
75%,2.020053e+07,17.25000,296543.250000,86349.000000,7905.250000,19499.000000,7904.750000,5379.500000,121476.000000,4996.750000,...,15162.750000,28786.000000,17261.250000,4906.750000,17915.750000,2182.250000,15947.000000,9750.500000,16780.500000,0.0
max,2.020053e+07,23.00000,439173.000000,150733.000000,19038.000000,50026.000000,17634.000000,14861.000000,226740.000000,10638.000000,...,21257.000000,48132.000000,26146.000000,15578.000000,35618.000000,5946.000000,29441.000000,19906.000000,31504.000000,0.0


## Dataloader
* 한 칼럼에 대한 7일(168행) 데이터를 input_data, 뒤따르는 7일 데이터를 output_data로 반환합니다.
* 도로별 차이를 두지 않고 모든 도로를 동일한 타입의 데이터로 취급합니다.
* 모든 csv 파일의 마지막 168행은 예측해야하는 값이므로 input으로 들어가지 않습니다.

In [10]:
class CustomDataset(data.Dataset):      # torch.utils.data.Dataset 클래스의 상속 클래스 CustomDataset class 생성. 상속 클래스 생성시 __init__, __getitem__, __len__함수는 기본적으로 정의해줘야 함.
    
    def __init__(self, root, seq_len, batch_size=64, phase='train'):      # 데이터 로드 단계에 사용될 여러 변수들을 'self.변수명'의 형태로 지정해두는 함수
        
        self.root = root      # CustomDataset 객체 생성 시 데이터 경로 앞부분(공통 부분)을 root로 입력받아 저장
        self.phase = phase      # CustomDataset 객체 생성 시 데이터 경로 뒷부분(train/validate/test)을 phase로 입력받아 저장
        self.label_path = os.path.join(self.root, self.phase + '.csv')      # 데이터 전체 경로 생성
        df = pd.read_csv(self.label_path)      # 생성한 데이터 전체 경로로부터 데이터 로드
        
        self.seq_len = seq_len * 24      # 일 단위 기간을 입력 받은 후 시간 단위 기간으로 변환하여 저장
        self.batch_size = batch_size
        self.labels = {}
        
        timestamps = [(i, j) for (i, j) in zip(list(df['날짜']), list(df['시간']))]      # 날짜와 시간 정보가 튜플로 들어 있는 리스트 생성
        categories = df.columns.values.tolist()[2:]      # 도로명 column list 생성

        input_data = []
        output_data = []

        for t in range(len(timestamps)):
            temp_input_data = []
            temp_output_data = []
            for col in categories:
                road = df[col].tolist()
                inp = [float(i) for i in road[t:t+self.seq_len]]      # input 데이터 시계열 구간 설정
                outp = [float(j) for j in road[t+self.seq_len:t+2*self.seq_len]]      # output 데이터 시계열 구간 설정
                temp_input_data.append(inp) 
                temp_output_data.append(outp)
            input_data.append(temp_input_data)
            output_data.append(temp_output_data)
            
# input_data : [[첫번째 input 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 input 기간 동안의 35번째 도로의 통행량 list], ...,
#               [마지막 input 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 input 기간 동안의 35번째 도로의 통행량 list]]
# output_data : [[첫번째 output 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 output 기간 동안의 35번째 도로의 통행량 list], ...,
#                [마지막 output 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 output 기간 동안의 35번째 도로의 통행량 list]]
        
        self.labels['timestamp'] = timestamps
        self.labels['category'] = categories
        self.labels['input'] = input_data
        self.labels['output'] = output_data

    def __getitem__(self, index):      # index를 가지고 데이터를 하나씩 불러올 수 있게 하는 함수

#         데이터 내 index가 부여되는 형태

#                 | road_1    road_2    ...  road_35
#                -------------------------------------
#         time_1  | index_0   index_1   ...  index_34
#         time_2  | index_35  index_36  ...  index_69

        row = index // 35      # index를 35(도로수)로 나눈 몫  ex) 71//35 -> 2
        col = index % 35      # index를 35(도로수)로 나눈 나머지  ex) 71%35 -> 1

        timestamp = self.labels['timestamp'][row]      # (날짜, 시간) 튜플이 들어있는 list에서 row번째 시점에 해당하는 튜플
        category = self.labels['category'][col]      # 도로명 column list에서 col번째 도로에 해당하는 element
        
        input_data = torch.tensor(self.labels['input'][row][col])      # input_data list에서, row번째 시점의 col번째 도로 교통량 정보

        if self.phase != 'test':
            output_data = torch.tensor(self.labels['output'][row][col])
        else:
            output_data = []

        return timestamp, category, (input_data, output_data)

    def __len__(self):      # getitem 함수를 통해 데이터를 불러오려면,전체 index 길이를 알아야 한다.
        return (len(self.labels['timestamp']) - (self.seq_len * 2) + 1) * 35      # 특정 시점이 아닌 특정 기간을 하나의 data 단위로 설정하면, 전체 샘플 수는 감소함을 반영 


def data_loader(root, phase='train', batch_size=64, seq_len=7, drop_last=False):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False

    dataset = CustomDataset(root, seq_len, batch_size, phase)
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

## Model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LSTMNet(nn.Module):
    def __init__(self,
                 input_size=168,      # input 길이는 168시간(7일 X 24시간)
                 hidden_size=1024,
                 output_size=168,      # output 길이는 168시간(7일 X 24시간)
                 batch_size=64,
                 num_layers=3,
                 dropout=0,
                 batch_first=False):      # batch_first(default=False) : 배치 차원을 첫번째 차원으로 하여 데이터를 불러올 것인지 여부

        super(LSTMNet, self).__init__()

        self.hidden_size = hidden_size
        
        ##### Layer 1
        self.lstm1 = nn.LSTM(input_size,
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Layer 2
        self.lstm2 = nn.LSTM(hidden_size, 
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Finalize
        self.linear = nn.Linear(hidden_size, 
                                output_size)
        
        self.activation = nn.LeakyReLU(0.2)

        
    def forward(self, x, h_in, c_in):

        h_in = nn.Parameter(h_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), h_in 이라는 이름의 파라미터 생성 
        c_in = nn.Parameter(c_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), c_in 이라는 이름의 파라미터 생성

        # Layer 1
        lstm_out, (h_1, c_1) = self.lstm1(x, (h_in, c_in))
        lstm_out = self.activation(lstm_out)

        # Layer2
        lstm_out, (h_2, c_2) = self.lstm2(lstm_out, (h_1, c_1))
        lstm_out = self.activation(lstm_out)

        # Final
        predictions = self.linear(lstm_out)
        
        return predictions, (h_2, c_2)

In [12]:
# 결과 파일과 모델 가중치 파일 저장을 위해 log 디렉토리 생성. 중요한 파일이 덮어씌워지지 않도록 주의
os.makedirs('log', exist_ok=True)


def save_model(model_name, model, optimizer):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, os.path.join('log', model_name + '.pth'))
    print('model saved\n')
    return os.path.join('log', model_name + '.pth')


def load_model(model_name, model, optimizer=None):
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded')

## Hyperparameters

In [13]:
dtype = torch.float
model_name = 'sequential'

batch_size = 64
num_epochs = 5
val_epoch = 1
base_lr = 0.01
seq_len = 7

input_size = seq_len * 24
output_size = input_size
hidden_size = 1024
num_layers = 6

## Training Setting

In [14]:
# model
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)
model = model.to(device)

# loss function
criterion = nn.MSELoss()

# optimizer
optimizer = Adam(model.parameters(), lr=base_lr)      # optimizer로는 Adam이 가장 무난합니다. Adam을 쓰면 learning_rate를 따로 지정해주지 않아도 알아서 조정됩니다.

In [15]:
print(model)

LSTMNet(
  (lstm1): LSTM(168, 1024, num_layers=6, dropout=0.2)
  (lstm2): LSTM(1024, 1024, num_layers=6, dropout=0.2)
  (linear): Linear(in_features=1024, out_features=168, bias=True)
  (activation): LeakyReLU(negative_slope=0.2)
)


In [17]:
# get data loader
train_dataloader = data_loader(root=DATASET_PATH,
                               phase='EDA_train',
                               batch_size=batch_size,
                               seq_len=seq_len,
                               drop_last=True)

validate_dataloader = data_loader(root=DATASET_PATH,
                                  phase='EDA_valid',
                                  batch_size=1,
                                  seq_len=seq_len,
                                  drop_last=True)

## Train

In [ ]:
train_batch_loss = 0.0
train_epoch_loss = 0.0

valid_epoch_loss = 0.0
valid_min_epoch_loss = np.inf

for epoch in range(num_epochs):

    model.train()      # 모델을 train mode로 전환. train mode일 때만 적용되어야 하는 drop out 등이 적용될 수 있게 하기 위함 

    for iter_, sample in enumerate(train_dataloader):      # enumerate 함수를 통해 train_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

        (h_in, c_in) = (torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device),
                        torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device))

        _, _, (input_data, output_data) = sample      # train_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. 학습에는 [[input_data], [output_data]]만 사용
        
        input_data = input_data.unsqueeze(0).to(device)
        output_data = output_data.unsqueeze(0).to(device)

        pred, (h_in, c_in) = model(input_data, h_in, c_in)
        
        loss = criterion(pred, output_data)

        model.zero_grad()    # 파라미터 업데이트는 batch 단위로 이루어지고, 매 batch마다 gradient를 초기화해주어야 함 
        loss.backward()      # backpropagation
        optimizer.step()      # 파라미터 업데이트
        
        train_batch_loss += loss.item()
        train_epoch_loss += loss.item()

        if iter_ % 400 == 399:      # 400개의 batch마다 training Loss 출력
            print('Train Epoch: {:2} | Batch: {:4} | Loss: {:1.2f}'.format(epoch, iter_+1, train_batch_loss/400))
            train_batch_loss = 0
            
    train_epoch_loss = 0.0

    
    model.eval()      # 모델을 eval mode로 전환. eval mode에서 적용되면 안되는 drop out 등이 적용되지 않게 하기 위함

    with torch.no_grad():      # validation / test set에 대해서는 weight 및 bias의 update, 즉, gradient descent가 일어나지 않도록 no_grad()를 선언
        (h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                        torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

        for iter_, sample in enumerate(validate_dataloader):      # enumerate 함수를 통해 validate_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

            _, _, (input_data, output_data) = sample      # validate_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. validation에는 [[input_data], [output_data]]만 사용

            input_data = input_data.unsqueeze(0).to(device)
            output_data = output_data.unsqueeze(0).to(device)

            pred, (h_in, c_in) = model(input_data, h_in, c_in)
            loss = criterion(pred, output_data)
            valid_epoch_loss += loss.item()

        print('\nValid Epoch: {:2} | Loss: {:1.2f}'.format(epoch, valid_epoch_loss/len(validate_dataloader)))

        if valid_epoch_loss < valid_min_epoch_loss:
            save_model('best', model, optimizer)
            valid_min_epoch_loss = valid_epoch_loss

        valid_epoch_loss = 0.0

Train Epoch:  0 | Batch:  400 | Loss: 4267865896.32
Train Epoch:  0 | Batch:  800 | Loss: 3000447685.12
Train Epoch:  0 | Batch: 1200 | Loss: 3294773683.84
Train Epoch:  0 | Batch: 1600 | Loss: 3932494653.44

Valid Epoch:  0 | Loss: 3677489726.57
model saved

Train Epoch:  1 | Batch:  400 | Loss: 4255893056.64
Train Epoch:  1 | Batch:  800 | Loss: 2795657255.68
Train Epoch:  1 | Batch: 1200 | Loss: 3086259799.04
Train Epoch:  1 | Batch: 1600 | Loss: 3624352547.84

Valid Epoch:  1 | Loss: 3345112721.09
model saved

Train Epoch:  2 | Batch:  400 | Loss: 3894452669.44
Train Epoch:  2 | Batch:  800 | Loss: 2522844839.36
Train Epoch:  2 | Batch: 1200 | Loss: 2787309860.16
Train Epoch:  2 | Batch: 1600 | Loss: 3343489224.00

Valid Epoch:  2 | Loss: 3113737948.80
model saved

Train Epoch:  3 | Batch:  400 | Loss: 3677062906.88
Train Epoch:  3 | Batch:  800 | Loss: 2403657257.28
Train Epoch:  3 | Batch: 1200 | Loss: 2669864300.16


## Inference

In [15]:
dtype = torch.float
seq_len = 7

input_size = seq_len * 24
hidden_size = 1024
output_size = input_size
batch_size = 1
num_layers = 6

In [16]:
test_dataloader = data_loader(root=DATASET_PATH,
                              phase='EDA_test',
                              batch_size=batch_size,
                              seq_len=seq_len,
                              drop_last=True)

In [17]:
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)

# model
model_name = 'log/best.pth'

load_model(model_name, model)
model = model.to(device)

model loaded


In [19]:
submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)

In [20]:
(h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

for iter_, sample in enumerate(test_dataloader):

    timestamp, category, (input_data, output_data) = sample
    input_data = input_data.unsqueeze(0).to(device)

    pred, (h_in, c_in) = model(input_data, h_in, c_in)

    for i, (t, h) in enumerate(zip(timestamp[0], timestamp[1])):
        for cat, row in zip(category, pred[0]):
            cat = f'{cat}'
            submission_table[cat] = row.tolist()

In [22]:
submission_table.to_csv('prediction.csv', index=False)